In [50]:
import tensorflow as tf
import os
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
tf.version.VERSION

'2.2.0'

In [3]:
def pretrained_embeddings(file_path, EMBEDDING_DIM, VOCAB_SIZE, word2idx):
    # 1.load in pre-trained word vectors     #feature vector for each word
    print("graph in function",tf.get_default_graph())   
    print('Loading word vectors...')
    word2vec = {}
    with open(os.path.join(file_path+'.%sd.txt' % EMBEDDING_DIM),  errors='ignore', encoding='utf8') as f:
        # is just a space-separated text file in the format:
        # word vec[0] vec[1] vec[2] ...
        for line in f:
            values = line.split()
            word = values[0]
            vec = np.asarray(values[1:], dtype='float32')
            word2vec[word] = vec

        print('Found %s word vectors.' % len(word2vec))

        # 2.prepare embedding matrix
        print('Filling pre-trained embeddings...')
        num_words = VOCAB_SIZE
        # initialization by zeros
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word2idx.items():
            if i < VOCAB_SIZE:
                embedding_vector = word2vec.get(word)
                if embedding_vector is not None:
                    # words not found in embedding index will be all zeros.
                    embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [4]:
file_path = "./glove.6B"
EMBEDDING_DIM = 100

In [5]:
file_path+'.%sd.txt' % EMBEDDING_DIM

'./glove.6B.100d.txt'

In [6]:
word2vec = {}

with open(os.path.join(file_path+'.%sd.txt' % EMBEDDING_DIM),  errors='ignore', encoding='utf8') as f:
    i = 0
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [7]:
def get_vector(word, w2v, embedding_dim):
    vec = np.zeros(embedding_dim)
    try:
        vec = w2v[word]
    except:
        pass
    return vec

In [8]:
get_vector('london', word2vec, 100)

array([ 6.0553e-01, -5.0886e-02, -1.5461e-01, -1.2327e-01,  6.6270e-01,
       -2.8506e-01, -6.8844e-01,  4.9135e-01, -6.8924e-01,  3.8926e-01,
        1.4359e-01, -4.8802e-01,  1.5746e-01,  8.3178e-01, -2.7923e-01,
        9.4755e-03, -1.1207e-01, -5.2099e-01, -3.7159e-01, -3.7951e-01,
        5.0083e-01, -3.4160e-01,  4.8098e-01, -1.1453e+00,  4.5958e-01,
       -6.5640e-01,  4.3018e-01, -4.2527e-01,  2.3089e-01,  7.8911e-01,
       -7.5434e-01,  1.0830e-01, -1.8071e-01, -5.5543e-04, -4.1071e-01,
        8.6157e-01,  5.3711e-02,  2.4208e-01, -2.6254e-01, -3.0915e-01,
       -2.9787e-01, -5.0758e-01, -2.9940e-01, -3.0442e-01,  7.3099e-01,
        1.4165e-01,  1.0339e-01, -2.9659e-01,  9.9400e-01, -4.1594e-01,
        3.8918e-01,  9.3532e-02,  1.0815e+00,  7.1774e-01, -1.1604e+00,
       -3.0277e+00, -9.2490e-01, -8.8455e-02,  6.1408e-01, -2.5770e-01,
       -2.6942e-01,  4.4647e-01, -8.3637e-01,  7.2481e-02,  3.0968e-02,
       -2.5574e-01, -2.4832e-01,  4.5399e-01,  6.7532e-01, -7.02

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("./data/Train.csv", sep=";")

In [3]:
dataset

,name,condition,opinion,rate,rate1
0,Zegerid,GERD,"""Using it as a replacement for Nexium, since i...",10,high
1,Ethosuximide,Seizures,"""This medicine is very good at controlling me ...",10,high
2,Tri-Sprintec,Birth Control,"""I just started taking Tri Sprintec after my l...",9,high
3,Levaquin,Pneumonia,"""This medicine made me feel absolutely horribl...",5,medium
4,Methylphenidate,ADHD,"""I&#039;ve been taking Concerta since 2003. Fo...",9,high
...,...,...,...,...,...
149995,Prozac,Anxiety and Stress,"""I was diagnosed with generalized anxiety and ...",9,high
149996,Scopolamine,Motion Sickness,"""Works great for motion sickness. Much more e...",10,high
149997,Epiduo,Acne,"""I&#039;ve been using Epiduo for about 8 month...",10,high
149998,Phillips' Milk of Magnesia,Constipation,"""Taking pain killers for shingles I was consta...",10,high


In [4]:
dataset['opinion'].str.replace('"', '').str.lower().str.replace('&#039;ve', ' have').str.replace('&#039;s', 'is')

dataset['opinion'] = dataset['opinion'].str.lower()
dataset['opinion'] = dataset['opinion'].str.replace('"', '')
dataset['opinion'] = dataset['opinion'].str.replace(',', '')
dataset['opinion'] = dataset['opinion'].str.replace('.', '')
dataset['opinion'] = dataset['opinion'].str.replace('!', '')
dataset['opinion'] = dataset['opinion'].str.replace(':', '')
dataset['opinion'] = dataset['opinion'].str.replace('&#039;ve', ' have')
dataset['opinion'] = dataset['opinion'].str.replace('&#039;s', ' is')
dataset['opinion'] = dataset['opinion'].str.replace('&#039;t', ' not')
dataset['opinion'] = dataset['opinion'].str.replace('&#039;m', ' am')
dataset['opinion'] = dataset['opinion'].str.replace('&#039;ll', ' will')


In [27]:
sample_opinion = dataset['opinion'][0] 

In [5]:
dataset

,name,condition,opinion,rate,rate1
0,Zegerid,GERD,using it as a replacement for nexium since ins...,10,high
1,Ethosuximide,Seizures,this medicine is very good at controlling me s...,10,high
2,Tri-Sprintec,Birth Control,i just started taking tri sprintec after my la...,9,high
3,Levaquin,Pneumonia,this medicine made me feel absolutely horrible...,5,medium
4,Methylphenidate,ADHD,i have been taking concerta since 2003 for me ...,9,high
...,...,...,...,...,...
149995,Prozac,Anxiety and Stress,i was diagnosed with generalized anxiety and p...,9,high
149996,Scopolamine,Motion Sickness,works great for motion sickness much more eff...,10,high
149997,Epiduo,Acne,i have been using epiduo for about 8 months no...,10,high
149998,Phillips' Milk of Magnesia,Constipation,taking pain killers for shingles i was constan...,10,high


In [6]:
dataset.drop_duplicates('opinion')

,name,condition,opinion,rate,rate1
0,Zegerid,GERD,using it as a replacement for nexium since ins...,10,high
1,Ethosuximide,Seizures,this medicine is very good at controlling me s...,10,high
2,Tri-Sprintec,Birth Control,i just started taking tri sprintec after my la...,9,high
3,Levaquin,Pneumonia,this medicine made me feel absolutely horrible...,5,medium
4,Methylphenidate,ADHD,i have been taking concerta since 2003 for me ...,9,high
...,...,...,...,...,...
149989,Tri-Lo-Sprintec,Birth Control,i had use bc before i stoped it because i was ...,1,low
149990,Ethinyl estradiol / levonorgestrel,Abnormal Uterine Bleeding,i didn not go on this pill as a birth control ...,6,medium
149992,Ethinyl estradiol / norethindrone,Birth Control,i have read almost all the reviews and i have ...,1,low
149993,Naproxen,Sciatica,i have had very big problems been taking norco...,10,high


In [28]:
def opinion_mean(opinion: str, w2v, embedding_dim):
    words = opinion.split()
    vec_sum = np.zeros(embedding_dim)
    number_of_words = 0
    for word in words:
        vec = get_vector(word, w2v, embedding_dim)
        if np.linalg.norm(vec-np.zeros(embedding_dim)) > 1e-5:
            number_of_words += 1
        vec_sum = vec_sum + vec
    return vec_sum / number_of_words

In [30]:
opinion_mean(sample_opinion, word2vec, 100)

array([-0.15427511,  0.12707239,  0.35868712, -0.18218694,  0.01455108,
        0.21512224, -0.07018973,  0.2298978 , -0.06927736,  0.04388645,
       -0.01925125, -0.04865904,  0.14613595,  0.00425707,  0.21716357,
       -0.26520777,  0.18001961, -0.03260566, -0.22419181,  0.16553172,
        0.31610323, -0.00798679,  0.14207322,  0.10039605,  0.23661318,
       -0.0170113 , -0.20278803, -0.48539128, -0.01207295, -0.1538183 ,
       -0.17432093,  0.46031325, -0.06569939, -0.02854962, -0.03786707,
        0.3628991 ,  0.01092459,  0.21116129,  0.00590539, -0.13569826,
       -0.26142451, -0.14836966,  0.20192877, -0.32625865,  0.02002101,
        0.10060166,  0.30492613, -0.31180021, -0.05516054, -0.70789443,
        0.04435397, -0.13992881,  0.18456231,  1.03101926, -0.27566989,
       -2.3161382 , -0.05861487, -0.25814567,  1.4924487 ,  0.35687561,
       -0.02166272,  0.65744   , -0.19823895,  0.1483947 ,  0.65397533,
       -0.024082  ,  0.45192134,  0.15319641,  0.20289282, -0.20

In [31]:
X_vectors = dataset['opinion'].apply(opinion_mean,args=(word2vec, 100))

In [41]:
X = np.stack(X_vectors.to_numpy())

In [43]:
X.shape

(150000, 100)

In [44]:
y = dataset['rate'].to_numpy()

In [53]:
y = y - 1

In [46]:
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers

In [47]:
simple_cnn = keras.Sequential()

simple_cnn.add(layers.Dense(128, activation='relu'))
simple_cnn.add(layers.Dense(128, activation='relu'))
simple_cnn.add(layers.Dense(10))

In [48]:
simple_cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [54]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.05, random_state=42)

In [55]:
simple_cnn.fit(X_train, y_train, epochs=30, validation_data=(X_validation, y_validation))

Epoch 1/30
4454/4454 [==============================] - 5s 1ms/step - loss: nan - accuracy: 0.1505 - val_loss: nan - val_accuracy: 0.1345
Epoch 2/30
4454/4454 [==============================] - 4s 855us/step - loss: nan - accuracy: 0.1344 - val_loss: nan - val_accuracy: 0.1345
Epoch 3/30
4454/4454 [==============================] - 5s 1ms/step - loss: nan - accuracy: 0.1344 - val_loss: nan - val_accuracy: 0.1345
Epoch 4/30
4454/4454 [==============================] - 6s 1ms/step - loss: nan - accuracy: 0.1344 - val_loss: nan - val_accuracy: 0.1345
Epoch 5/30
4454/4454 [==============================] - 5s 1ms/step - loss: nan - accuracy: 0.1344 - val_loss: nan - val_accuracy: 0.1345
Epoch 6/30
4454/4454 [==============================] - 4s 888us/step - loss: nan - accuracy: 0.1344 - val_loss: nan - val_accuracy: 0.1345
Epoch 7/30
4454/4454 [==============================] - 4s 849us/step - loss: nan - accuracy: 0.1344 - val_loss: nan - val_accuracy: 0.1345
Epoch 8/30
4454/4454 [======

KeyboardInterrupt: 